In [122]:
import sys
import sqlite3
import json
from operator import itemgetter
import numpy as np

In [13]:
dbname = '../db/sqlite3.db'
conn = sqlite3.connect(dbname)
conn.row_factory = sqlite3.Row
c = conn.cursor()

In [19]:
with open('../data/working/setting.json') as f:    
    data = json.load(f)

In [22]:
divideSize = data["grid"]["divideSize"]
northEast = data["grid"]["northEast"]
southWest = data["grid"]["southWest"]

In [94]:
def createGridPoints():
    dividerWidth = {
        "lat": (northEast["lat"] - southWest["lat"])/divideSize,
        "lng": (northEast["lng"] - southWest["lng"])/divideSize
    }
    
    def helper(i, j):
        return {
            "north": northEast["lat"] - dividerWidth["lat"]*i,
            "south": northEast["lat"] - dividerWidth["lat"]*(i+1),
            "east": northEast["lng"] - dividerWidth["lng"]*j,
            "west": northEast["lng"] - dividerWidth["lng"]*(j+1)
        }
    
    outer = []
    for i in range(divideSize):
        inner = []
        for j in range(divideSize):
            inner.append(helper(i, j))
        outer.append(inner)
        
    return outer

In [95]:
gridPoints = createGridPoints()

In [97]:
def findGridIndex(position, gridPoints):
    for i in range(divideSize):
        bounds = gridPoints[i][0]
        if bounds["south"] > position["lat"]:  
            continue            
        for j in range(divideSize):
            bounds = gridPoints[i][j]
            if bounds["west"] <= position["lng"]:
                return i*divideSize + j

In [39]:
c.execute("select * from cans where target = \"021021K1KAm\"")

markers = []
for row in c:
    lat, lng = row["GPSLatitude"], row["GPSLongtitude"]
    lngW = lng - lat*0.000046038 - lng*0.000083043 + 0.010040
    latW = lat - lat*0.00010695 + lng*0.000017464 + 0.0046017
    markers.append((row["id"], row["FrameIndex"], { "lat": latW, "lng": lngW })) 

In [118]:
gridIndices = [sys.maxint for _ in range(divideSize*divideSize)]

In [119]:
for marker in markers:
    id, frame, position = marker
    index = findGridIndex(position, gridPoints)
    if gridIndices[index] > frame:
        gridIndices[index] = frame


In [128]:
result = []
for i, value in enumerate(gridIndices):
    if value == sys.maxint:
        result.append((i, 0))
    else:
        result.append((i, value))

In [130]:
with open('../data/working/route.csv', 'w') as fp:
    fp.write('index,route\n')
    fp.write('\n'.join('%d,%d' % x for x in result))